In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'

In [2]:
from pathlib import Path
from zipfile import ZipFile
from dataclasses import dataclass, field
from typing import overload


@dataclass(slots=True)
class ImageData:
    directory_path: Path = field(default_factory=Path)
    
    image_paths: list[Path] = field(init=False)
    
    def __post_init__(self) -> None:
        with ZipFile(self.directory_path, 'r') as zip_file:
            name = zip_file.namelist()[0]
            zip_file.extractall(Path('./data/') / self.directory_path.parent)
        
        self.image_paths = list(
            self.directory_path.parent.glob(f'{name}/[!.]*')
        )
    
    @overload
    def __getitem__(self, idx: int) -> np.ndarray:
        return cv2.imread(str(self.image_paths[idx]), cv2.IMREAD_GRAYSCALE)
    
    def __getitem__(self, idx: str) -> np.ndarray:
        return cv2.imread(self.directory_path.parent / idx, cv2.IMREAD_GRAYSCALE)


In [3]:
aloes = ImageData(Path('aloes.zip'))
example = ImageData(Path('example.zip'))
pairs = ImageData(Path('pairs.zip'))

In [ ]:
from typing import Tuple


def calibrate_camera(data: ImageData) -> None:
    # termination criteria
    criteria: Tuple[int, int, float] = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    calibration_flags: int = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_FIX_SKEW

    # inner size of chessboard
    width: int = 9
    height: int = 6

    # 0.025 meters
    square_size: float = 0.025
    
    # prepare object points, like (0, 0, 0), (1, 0, 0), (2, 0, 0), ..., (8, 6, 0)
    objp = np.zeros(( height * width, 1, 3), np.float64)
    objp [:, 0, :2] = np.mgrid[0: width, 0: height ].T.reshape(-1, 2)
    objp = objp * square_size # Create real world coords. Use your metric.
    # Arrays to store object points and image points from all the images .
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane .
    img_width = 640
    img_height = 480
    image_size = (img_width, img_height)
    path = ""
    image_dir = path + "pairs/"
    number_of_images = 50
    for i in range (1, number_of_images):
        # read image
        img = cv2.imread (image_dir + " left_%02d.png " % i)
        gray = cv2.cvtColor (img , cv2. COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        Y, X, channels = img.shape
        # skip images where the corners of the chessboard are too close to the edges of the image
        if ret:
            minRx = corners[: ,: ,0].min()
            maxRx = corners[: ,: ,0].max()
            minRy = corners[: ,: ,1].min()
            maxRy = corners[: ,: ,1].max()
            border_threshold_x = X /12
            border_threshold_y = Y /12
            x_thresh_bad = False
            if ( minRx < border_threshold_x ):
                x_thresh_bad = True
            y_thresh_bad = False
            if ( minRy < border_threshold_y ):
                y_thresh_bad = True
            if ( y_thresh_bad == True ) or ( x_thresh_bad == True ):
                continue
            # If found , add object points , image points ( after refining them )
            if ret == True :
                objpoints . append ( objp )
                # improving the location of points (sub - pixel )
                corners2 = cv2. cornerSubPix (gray , corners , (3, 3) , (-1, -1) , criteria )
                imgpoints . append ( corners2 )
                # Draw and display the corners
                # Show the image to see if pattern is found ! imshow function .
                cv2 . drawChessboardCorners (img , (width , height ), corners2 , ret )
                cv2 . imshow (" Corners ", img )
                cv2 . waitKey (5)
            else:
                print (" Chessboard couldn ’t detected . Image pair : ", i)
                continue